In [1]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [3]:
# Load the dataset
file_path = '/content/election 24 dataset.xlsx'
data = pd.read_excel(file_path)

In [4]:
# Display the first few rows of the dataset
data.head()

,Candidate Name,Political Party,EVM Votes,Postal Vote,Total Vote,% of Votes,Constituency,Status,State,Constituency No
0,K. C VENUGOPAL,Indian National Congress,398246,6314,404560,38.21,Alappuzha,Won,Kerala,11
1,A. M ARIFF,Communist Party of India (Marxist),335596,5451,341047,32.21,Alappuzha,Lost,Kerala,11
2,SOBHA SURENDRAN,Bharatiya Janata Party,295841,3807,299648,28.30,Alappuzha,Lost,Kerala,11
3,MURALEEDHARAN KONCHERILLAM,Bahujan Samaj Party,1578,48,1626,0.15,Alappuzha,Lost,Kerala,11
4,ARJUNAN,Socialist Unity Centre Of India (COMMUNIST),1301,18,1319,0.12,Alappuzha,Lost,Kerala,11


In [5]:
# Pie chart for percentage of votes per party
pie_chart = px.pie(data, values='% of Votes', names='Political Party',
                   title='Percentage of Votes by Political Party')

In [6]:
# Show the plot
pie_chart.show()

In [7]:
# Calculate the number of seats won by each party
seats_won = data[data['Status'] == 'Won'].groupby('Political Party').size().reset_index(name='Seats Won')

# Get top 5 candidates by total votes
top_5_candidates = data.nlargest(5, 'Total Vote')

In [8]:
# Initialize the Dash app
app = Dash(__name__)

In [9]:
# Layout of the Dash app
app.layout = html.Div([
    html.H1("Election 2024 Dashboard"),

    # Dropdown for selecting a constituency
    dcc.Dropdown(
        id='constituency-dropdown',
        options=[{'label': const, 'value': const} for const in data['Constituency'].unique()],
        value=data['Constituency'].unique()[0]
    ),

    # Bar chart for seats won by each party
    dcc.Graph(id='bar-chart-seats'),

    # Bar chart for top 5 candidates by total votes
    dcc.Graph(id='bar-chart-top-5'),

    # Pie chart for percentage of votes by party
    dcc.Graph(id='pie-chart-votes'),

    # Text output for the winning candidate
    html.Div(id='winning-candidate')
])


In [10]:
# Callback to update the charts and text based on the selected constituency
@app.callback(
    [Output('bar-chart-seats', 'figure'), Output('bar-chart-top-5', 'figure'), Output('pie-chart-votes', 'figure'), Output('winning-candidate', 'children')],
    [Input('constituency-dropdown', 'value')]
)
def update_dashboard(selected_constituency):
    # Bar chart for seats won by each party
    bar_chart_seats = px.bar(seats_won, x='Political Party', y='Seats Won', color='Political Party',
                             title='Seats Won by Each Party', labels={'Seats Won': 'Seats Won'})

    # Bar chart for top 5 candidates by total votes
    bar_chart_top_5 = px.bar(top_5_candidates, x='Candidate Name', y='Total Vote', color='Political Party',
                             title='Top 5 Candidates by Total Votes', labels={'Total Vote': 'Total Votes'})

    # Filter data for the selected constituency
    filtered_data = data[data['Constituency'] == selected_constituency]

    # Pie chart for percentage of votes by party
    pie_chart_votes = px.pie(filtered_data, values='% of Votes', names='Political Party',
                             title=f'Percentage of Votes by Party in {selected_constituency}')

    # Get the winning candidate in the selected constituency
    winning_candidate = filtered_data[filtered_data['Status'] == 'Won']['Candidate Name'].values[0]
    winning_party = filtered_data[filtered_data['Status'] == 'Won']['Political Party'].values[0]
    winning_text = f"The winning candidate in {selected_constituency} is {winning_candidate} from {winning_party}."

    return bar_chart_seats, bar_chart_top_5, pie_chart_votes, winning_text

In [11]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>